### 1. Set Up Neo4j
Install Neo4j Desktop or use Neo4j Aura for a cloud-based setup.
Use the official Neo4j Python driver (neo4j) to connect to your database.
### 2. Connect to Neo4j

In [1]:
import os
from dotenv import load_dotenv
from neo4j import GraphDatabase

# Load environment variables from the .env file
load_dotenv()

# Get Neo4j connection details from environment variables
uri = os.getenv('NEO4J_URI')
username = os.getenv('NEO4J_USERNAME')
password = os.getenv('NEO4J_PASSWORD')

# Create a driver instance using the credentials from .env
driver = GraphDatabase.driver(uri, auth=(username, password))

# Test connection
try:
    with driver.session() as session:
        result = session.run("RETURN 1")
        print("Connection to Neo4j established successfully.")
except Exception as e:
    print(f"Failed to connect to Neo4j: {e}")


### 3. Create Nodes and Relationships

In [7]:
def create_employee(tx, employee_id, name):
    tx.run("CREATE (e:Employee {id: $employee_id, name: $name})",
           employee_id=employee_id, name=name)

def create_department(tx, department_id, name):
    tx.run("CREATE (d:Department {id: $department_id, name: $name})",
           department_id=department_id, name=name)

def add_employee_to_department(tx, employee_id, department_id):
    tx.run("""
        MATCH (e:Employee {id: $employee_id})
        MATCH (d:Department {id: $department_id})
        CREATE (e)-[:BELONGS_TO]->(d)
        """, employee_id=employee_id, department_id=department_id)

with driver.session() as session:
    session.write_transaction(create_employee, "emp1", "John Doe")
    session.write_transaction(create_department, "dept1", "Engineering")
    session.write_transaction(add_employee_to_department, "emp1", "dept1")


C:\Users\Rohith\AppData\Local\Temp\ipykernel_17384\1616732719.py:17: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_employee, "emp1", "John Doe")
C:\Users\Rohith\AppData\Local\Temp\ipykernel_17384\1616732719.py:18: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_department, "dept1", "Engineering")
C:\Users\Rohith\AppData\Local\Temp\ipykernel_17384\1616732719.py:19: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(add_employee_to_department, "emp1", "dept1")


### 4. Retrieve Employees in a Specific Department

In [8]:
def get_employees_in_department(tx, department_name):
    result = tx.run("""
        MATCH (e:Employee)-[:BELONGS_TO]->(d:Department {name: $department_name})
        RETURN e.name AS employee_name
        """, department_name=department_name)
    return [record["employee_name"] for record in result]

with driver.session() as session:
    employees = session.read_transaction(get_employees_in_department, "Engineering")
    print("Employees in Engineering:", employees)


Employees in Engineering: ['John Doe', 'John Doe']


C:\Users\Rohith\AppData\Local\Temp\ipykernel_17384\3093131053.py:9: DeprecationWarning: read_transaction has been renamed to execute_read
  employees = session.read_transaction(get_employees_in_department, "Engineering")


### 5. Update an Employee's Department

In [4]:
def update_employee_department(tx, employee_id, new_department_id):
    tx.run("""
        MATCH (e:Employee {id: $employee_id})-[r:BELONGS_TO]->(d:Department)
        DELETE r
        WITH e
        MATCH (new_d:Department {id: $new_department_id})
        CREATE (e)-[:BELONGS_TO]->(new_d)
        """, employee_id=employee_id, new_department_id=new_department_id)

with driver.session() as session:
    session.write_transaction(update_employee_department, "emp1", "dept2")


C:\Users\Rohith\AppData\Local\Temp\ipykernel_17384\3536430136.py:11: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(update_employee_department, "emp1", "dept2")


### 6. Remove an Employee

In [5]:
def remove_employee(tx, employee_id):
    tx.run("MATCH (e:Employee {id: $employee_id}) DETACH DELETE e",
           employee_id=employee_id)

with driver.session() as session:
    session.write_transaction(remove_employee, "emp1")


C:\Users\Rohith\AppData\Local\Temp\ipykernel_17384\4056894984.py:6: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(remove_employee, "emp1")


### Considerations and Advantages
* **Advantages**: Neo4j is highly suited for hierarchical data due to its native graph structure, which allows for efficient queries on complex relationships and traversals, such as finding all employees in a department or traversing up the organizational hierarchy.

* **Scalability**: For large-scale graphs, Neo4j offers horizontal scaling via sharding and clustering, making it suitable for handling extensive graph data sets.